# Student : Arora, Sanjana (V00966221)

In [ ]:
import pandas as pd
import numpy as np

Please upload elections_clean.csv provided within the zip folder

In [ ]:
df = pd.read_csv('elections_clean.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'votes', 'UnemploymentRate2015', 'MedHHInc2014',
       'PerCapitaInc', 'PovertyAllAgesPct2014', 'Deep_Pov_All', 'Population',
       'Area in square miles - Total area', 'PopDensity', 'TOT_MALE_rate',
       'TOT_FEMALE_rate', 'voter_turnout_rate', 'Democrat', 'State', 'County',
       'Education', 'Religion', 'Old', 'Young', 'Adult', 'EthnicMale',
       'EthnicFemale'],
      dtype='object')

In [ ]:
subset = df[['Education', 'Religion', 'EthnicMale', 'EthnicFemale', 'Democrat']].copy()

In [ ]:
subset.columns

Index(['Education', 'Religion', 'EthnicMale', 'EthnicFemale', 'Democrat'], dtype='object')

In [ ]:
subset.head()

,Education,Religion,EthnicMale,EthnicFemale,Democrat
0,OnlyDiploma,Other Misc,WHITE_MALE_rate,WHITE_FEMALE_rate,0
1,OnlyDiploma,Catholic,WHITE_MALE_rate,WHITE_FEMALE_rate,0
2,College,Christian Generic,WHITE_MALE_rate,WHITE_FEMALE_rate,0
3,OnlyDiploma,Catholic,WHITE_MALE_rate,WHITE_FEMALE_rate,0
4,College,Catholic,WHITE_MALE_rate,WHITE_FEMALE_rate,0


In [ ]:
#Calculating the entropy

def entropy(Y): # Y is the Target Columns
  values, counts = np.unique(Y,return_counts = True)
  for i in range(len(values)):
    prob = counts[i]/np.sum(counts) # calculating the probability
    h = np.sum(-(prob)*np.log2(prob))
  return h

In [ ]:
#Calculating the Information Gain
def InfoGain(dataset,split_feature_name, Y):
  total_h = entropy(dataset[Y]) 
  values, counts = np.unique(dataset[split_feature_name],return_counts = True)
  weighted_h = 0
  for i in range(len(values)):
    prob = counts[i]/np.sum(counts)
    feature = dataset.where(dataset[split_feature_name]==values[i]).dropna()[Y]
    h = entropy(feature)
    weighted_h = weighted_h + prob*h
  IG = total_h - weighted_h
  return IG

DecisionClassifier function takes the following input parameters:
1. Dataset (This reduces are we recurse in this function)
2. Original Dataset (The Training/Validation Dataset for out classification)
3. Filter_node_class (The values of target vector (Democrat) that called the DecisionClassifier recursion
4. Depth (The max depth of the tree- User can change this value for pruning the tree)
5. Length (parameter used by the function to calculate the max depth of the tree 

DecisionClassifier function outputs the following input parameters:
1. Tree
2. Names of the feature appearing in decision stump
3. Max depth of the tree

In [ ]:
def DecisionClassifier(dataset,originaldata,features,Y,depth=20,filter_node_class = None,height=-1, length=[], feat=[]):
      
      #Storing and Adding the length of the tree
      height=height
      length.append(height)
      
      # Checking if all target values have same values left
      if (len(np.unique(dataset[Y])) <= 1):
          
          return np.unique(dataset[Y])[0], length, feat # return the value and length of the tree
      
      # if dataset has got empty
      elif (len(dataset)==0 or height==depth) :
        
        # index of the mode target feature value in dataset
        index = np.argmax(np.unique(originaldata[Y],return_counts=True)[1])
        mode = np.unique(originaldata[Y])[index]
        return mode, length,feat #returning the target feature value occuring maximum number of times in original dataset
      
      # if the number of features have got zero, returning the mode target feature value indicated by the parent node that called the DecisionClassifier recursion
      elif (len(features) ==0):
        return filter_node_class, length,feat
      
      
      # if none of the conditions are true, growing the tree
      else:
        
        # growing the tree
        # assigning parent node class feature values as the selected target feature value having max. no. of occurrences
          index = np.argmax(np.unique(dataset[Y],return_counts=True)[1])
          filter_node_class  = np.unique(dataset[Y])[index]
          
         # selecting the feature which best splits the dataset 
          item_vals = [InfoGain(dataset,feature,Y) for feature in features]
          
          # best feature is the feature having maximum information gain about the target feature
          best_featureindex = np.argmax(item_vals)
          best_feature = features[best_featureindex]
         
          # tree structure
          tree = {best_feature:{}}
          feat.append(best_feature)
          # iterating over all features except for the best feature
          features = [i for i in features if i != best_feature]
        # Adding the height as a feature is getting added to the decision stump
          height+= 1
          
        
          
            #finding the dataset corresponding the best feature for further analysis
          for vals in np.unique(dataset[best_feature]):
              
              sub_data = dataset.where(dataset[best_feature] == vals).dropna()  
              subtree, length, feat = DecisionClassifier(sub_data,dataset,features,Y,depth,filter_node_class,height=height, length=length,feat=feat)
            
            
            # adding subtree, under the grown tree
              tree[best_feature][vals] = subtree
          return tree, length, feat     

In [ ]:
def prediction(query,tree,default = 1):
    for key in list(query.keys()):
        #check for every key in the query, if the feature exists in the keys of the tree
        if key in list(tree.keys()):
            
            try:
                result = tree[key][query[key]] 
            except:
              #if the feature does not exist return default value
                return default
  
            
            result = tree[key][query[key]]
            
            if isinstance(result,dict):
                return prediction(query,result)

            else:
                return result

In [ ]:
def error_calc(data,tree,target_attribute_name):
    # generating new queries as dictionaries from the dataset that is devoid of the target feature
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    # dataframe capturing predicted target values
    predicted = pd.DataFrame(columns=["predicted"])  

    #Calculate the prediction accuracy and error rates
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = prediction(queries[i],tree,1)
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data[target_attribute_name])/len(data))*100,'%')
    print('The Mean Absolute Error: ', np.mean(np.square(predicted["predicted"] - np.array(data[target_attribute_name])))) 

**Function for Splitting the dataset into testing and training**

In [ ]:
def train_validation_split(dataset):
    validation_split = .3
    dataset_size = len(dataset)
    split = int(np.floor(validation_split * dataset_size))
    training_data = dataset.iloc[split:].reset_index(drop=True)
    validation_data = dataset.iloc[:split].reset_index(drop=True)
    return training_data,validation_data

**Shuffling the dataset and splitting the dataset into training and validation**

In [ ]:
subset = subset.sample(frac = 1)
training_data = train_validation_split(subset)[0]
validation_data = train_validation_split(subset)[1] 
print(training_data.shape,validation_data.shape )

(2202, 5) (943, 5)


Q2. **Implementing the decision tree from scratch **

User can input desired depth along with other parameters in DecisionClassifier

Maximum Depth of the tree is 3

In [ ]:
tree, length,feat = DecisionClassifier(training_data,training_data,training_data.columns[:-1],'Democrat')
import pprint
pprint.pprint(tree)
print('The max depth of the tree is:', max(length))


{'Education': {'BachelorOrHigher': {'Religion': {'Amish': 0.0,
                                                 'Catholic': {'EthnicFemale': {'BLACK_FEMALE_rate': 1.0,
                                                                               'WHITE_FEMALE_rate': {'EthnicMale': {'ASIAN_MALE_rate': 1.0,
                                                                                                                    'WHITE_MALE_rate': 1.0}}}},
                                                 'Christian Generic': {'EthnicFemale': {'BLACK_FEMALE_rate': 1.0,
                                                                                        'WHITE_FEMALE_rate': {'EthnicMale': {'WHITE_MALE_rate': 0.0}}}},
                                                 'Other': {'EthnicMale': {'WHITE_MALE_rate': {'EthnicFemale': {'WHITE_FEMALE_rate': 0.0}}}}}},
               'College': {'EthnicFemale': {'BLACK_FEMALE_rate': {'EthnicMale': {'BLACK_MALE_rate': 1.0,
                                 

Number of feature repeating in decision stumps

In [ ]:
Names = np.unique(feat,return_counts=True)[0]
Number = np.unique(feat,return_counts=True)[1]
print('The repeating features in decision stump:', Names, 'in the respective order of:', Number)

The repeating features in decision stump: ['Education' 'EthnicFemale' 'EthnicMale' 'Religion'] in the respective order of: [ 1  7 13  9]


Training Accuracy & Error

In [ ]:
# Training Error
error_calc(training_data,tree,'Democrat')

The prediction accuracy is:  91.46230699364214 %
The Mean Absolute Error:  0.08537693006357856


Testing Accuracy & Error

In [ ]:
# Validation Error
error_calc(validation_data,tree,'Democrat')

The prediction accuracy is:  90.98621420996818 %
The Mean Absolute Error:  0.09013785790031813


**Bonus Q1 Making more categorical related features**

In [ ]:
df['Age'] = df[[ 'Old', 'Young', 'Adult']].idxmax(axis=1)

#binning income related features
bins = [0,1, 2, 3, 4, 5]
df['binned_Hincome'] = pd.qcut(df['MedHHInc2014'], q= 6, labels=bins)
df['binned_PerCincome'] = pd.qcut(df['PerCapitaInc'], q= 6, labels=bins)
df.columns

Index(['Unnamed: 0', 'votes', 'UnemploymentRate2015', 'MedHHInc2014',
       'PerCapitaInc', 'PovertyAllAgesPct2014', 'Deep_Pov_All', 'Population',
       'Area in square miles - Total area', 'PopDensity', 'TOT_MALE_rate',
       'TOT_FEMALE_rate', 'voter_turnout_rate', 'Democrat', 'State', 'County',
       'Education', 'Religion', 'Old', 'Young', 'Adult', 'EthnicMale',
       'EthnicFemale', 'Age', 'binned_Hincome', 'binned_PerCincome'],
      dtype='object')

In [ ]:
subset1 = df[['Education', 'Religion', 'EthnicMale', 'EthnicFemale','Age', 'binned_Hincome', 'binned_PerCincome', 'Democrat']].copy()

In [ ]:
subset1 = subset1.sample(frac = 1)
training_data1 = train_validation_split(subset1)[0]
validation_data1 = train_validation_split(subset1)[1] 
print(training_data1.shape,validation_data1.shape )

(2202, 8) (943, 8)


In [ ]:
#Training the decision tree with new data
tree_new, length_new, feat_new = DecisionClassifier(training_data1,training_data1,training_data1.columns[:-1],'Democrat')

In [ ]:
print('The max depth of the tree is:', max(length_new))

The max depth of the tree is: 6


In [ ]:
# Training Error
error_calc(training_data1,tree_new,'Democrat')

The prediction accuracy is:  92.41598546775658 %
The Mean Absolute Error:  0.07584014532243415


In [ ]:
# Validation Error
error_calc(validation_data1,tree_new,'Democrat')

The prediction accuracy is:  90.66808059384942 %
The Mean Absolute Error:  0.09331919406150584


By adding more categories, the training error changed from approx. 0.085 to 0.075, while testing error increased slightly by 0.090 to 0.093.

The length of the tree increased from 3 to 6

Referred Lecture slides and https://www.analyticsvidhya.com/blog/2016/04/tree-based-algorithms-complete-tutorial-scratch-in-python/